# AML — Task 1
## Predict the age of a brain from MRI features
---

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
from pandas_profiling import ProfileReport

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest as IF

In [3]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

## Import datasets

In [4]:
def import_data():
    X_train = pd.read_csv('data/X_train.csv').drop(columns=['id'])
    y_train = pd.read_csv('data/y_train.csv').drop(columns=['id'])
    X_test = pd.read_csv('data/X_test.csv').drop(columns=['id'])
    return X_train, y_train, X_test

---
## Exporting Data as csv

In [5]:
def export_csv(X_train_cleaned, y_train_cleaned, X_test_cleaned):
    X_train_cleaned.to_csv('X_train_cleaned.csv', index=False)
    y_train_cleaned.to_csv('y_train_cleaned.csv', index=False)
    X_test_cleaned.to_csv('X_test_cleaned.csv', index=False)

---
## Task 1: Outlier detection

In [6]:
"""Remove the ouliers from our dataset. Replace temporarily the Nan values by the mean to perform outlier selection

Parameters
----------
X_train : pd.df
    The features (what we will use to see the outliers)
y_train : pd.df
    The labels
contamination : int, optional
    The percent of outliers found by the isolation forest if it's used.
    
Return
------
(pd.df, pd.df)
    The data with the outliers rows removed
"""
def remove_outliers(X_train, y_train, contamination='auto'):
    # Save a mask of the imputed values to be able to redo the imputation once the outlier detection is done
    X_train_null_mask = np.array(X_train.isna())
    
    # Need to impute nan values for the outlier detection to work (cannot deal with nan)
    X_train = SimpleImputer(strategy="median", verbose=1).fit_transform(X_train)
    
    outliers_mask=IForest(X_train,contamination=contamination)
    
    #Replace the Nan values (The outlier detection shouldn't replace NaN values by itself)
    X_train = pd.DataFrame(X_train).mask(X_train_null_mask, other=np.NaN, inplace=False)
    
    # Remove outliers from the training set
    X_train = np.array(X_train)[outliers_mask == 1, :]
    y_train = np.array(y_train)[outliers_mask == 1, :]
    
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)
    
    return (X_train, y_train)

### Method 1: Isolation Forest

In [7]:
"""Use Isolation Forest to compute mask of outliers

Parameters
----------
X_train : pd.df
    The features
contamination : int, optional
    The percent of outliers found by the isolation forest if it's used.
verbose : int, optional
    If not set to 0, print messages

Return
------
pd.Series
    The mask with 0 for the outliers and 1 for the normal values
"""
def IForest(X_train, contamination='auto', verbose=1):
    clf = IF(contamination=contamination, random_state=0, verbose=verbose).fit(X_train)
    outliers_mask = pd.Series(clf.predict(X_train)).map({1:1, -1:0}) #Mask with 0 for outliers and 1 for non outliers
    
    if verbose:
        print(f"Detected {(outliers_mask == 0).sum()} outliers, out of {outliers_mask.shape[0]} samples ({100 * (outliers_mask == 0).sum() / outliers_mask.shape[0]:.2f}%).")
    
    return outliers_mask

---
## Task 2: Data scaling
Done as soon as possible because can have an effect (e.g. on distances for `KNNImputer`)

In [8]:
def scale(X_train_no_outliers, X_test):
    # Do the scaling, saving the scaler to use it for X_test too. No need imputation, just ignore Nan values
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train_no_outliers))
    X_test_scaled = pd.DataFrame(scaler.transform(X_test))
    return (X_train_scaled, X_test_scaled)

---
## Task 3: impute values

In [9]:
def impute_values(X_train_scaled, X_test_scaled):
    print(f"For the train dataset, there are {np.array(X_train_scaled.isna()).sum().sum()} nan values, out of {X_train_scaled.shape[0]*X_train_scaled.shape[1]} ({100*np.array(X_train_scaled.isna()).sum().sum()/(X_train_scaled.shape[0]*X_train_scaled.shape[1]):.2f}%).")
    imputer = KNNImputer(n_neighbors=8, weights='uniform').fit(X_train_scaled)
    X_train_imputed = pd.DataFrame(imputer.transform(X_train_scaled))
    X_test_imputed = pd.DataFrame(imputer.transform(X_test_scaled))
    return (X_train_imputed, X_test_imputed)

---
## Task 4: Feature selection

In [10]:
def select_features(X_train_imputed, X_test_imputed):
    X_train_selected_features, X_test_selected_features = remove_constant_features(X_train_imputed, X_test_imputed)
    X_train_selected_features, X_test_selected_features = remove_too_coorelated_features(X_train_selected_features, X_test_selected_features)
    X_train_selected_features, X_test_selected_features = remove_too_uncoorelated_features(X_train_selected_features, X_test_selected_features)

    return X_train_selected_features, X_test_selected_features

In [11]:
def remove_constant_features(X_train_imputed, X_test_imputed, verbose=1):
    X_train_selected_features = X_train_imputed.loc[:, (X_train_imputed != X_train_imputed.iloc[0]).any()]
    X_test_selected_features = X_test_imputed.loc[:, (X_train_imputed != X_train_imputed.iloc[0]).any()]
    
    if verbose:
        print(f"{X_train_imputed.shape[1]-X_train_selected_features.shape[1]} features removed because of constant values ({100*(X_train_imputed.shape[1]-X_train_selected_features.shape[1])/X_train_imputed.shape[1]:.2f}%)")
    
    return X_train_selected_features, X_test_selected_features

In [12]:
def remove_too_coorelated_features(X_train, X_test, threshold=0.8, verbose=1):
    X_train_corr_ = X_train.corr()

    X_train_too_correlated = (X_train_corr_.mask(
        np.tril(np.ones([len(X_train_corr_)]*2, dtype=bool))).abs() > threshold).any()
    
    
    X_train_selected_features = X_train.loc[:, (~X_train_too_correlated)]
    X_test_selected_features = X_test.loc[:, (~X_train_too_correlated)]
    
    if verbose:
        print(f"{X_train.shape[1]-X_train_selected_features.shape[1]} features removed because of correlation > {threshold} ({100*(X_train.shape[1]-X_train_selected_features.shape[1])/X_train.shape[1]:.2f}%)")

    return X_train_selected_features, X_test_selected_features

In [13]:
def remove_too_uncoorelated_features(X_train, X_test, threshold=0.08, verbose=1):
    X_train_corr_ = X_train.corr()

    X_train_too_uncorrelated = (X_train_corr_.replace(1, 0).abs() < threshold).all()
    
    X_train_selected_features = X_train.loc[:, (~X_train_too_uncorrelated)]
    X_test_selected_features = X_test.loc[:, (~X_train_too_uncorrelated)]
    
    if verbose:
        print(f"{X_train.shape[1]-X_train_selected_features.shape[1]} features removed because of correlation < {threshold} ({100*(X_train.shape[1]-X_train_selected_features.shape[1])/X_train.shape[1]:.2f}%) with every other features")

    return X_train_selected_features, X_test_selected_features

---
## Task 5 Models

In [14]:
def model(X_train_cleaned, y_train_cleaned):
    gs_lasso = Lasso_model(X_train_cleaned, y_train_cleaned)
    gs_svr = SVR_model(X_train_cleaned, y_train_cleaned)
    
    return gs_lasso if gs_lasso.best_score_ > gs_svr.best_score_ else gs_svr

### Model 1: Lasso

In [15]:
def Lasso_model(X_train_cleaned, y_train_cleaned):
    lasso = Lasso(max_iter=100000)
    gs_lasso_params = {
    'alpha': np.logspace(-1, 0, 20),
    }
    gs_lasso = GridSearchCV(lasso, gs_lasso_params, cv=5, verbose=3)
    gs_lasso.fit(X_train_cleaned, y_train_cleaned)
    print(f"The best validation score obtained is {gs_lasso.best_score_:.5f} with\n\talpha: {gs_lasso.best_params_['alpha']}")
    return gs_lasso;

### Model 2: SVR (SVM for regression)

In [16]:
def SVR_model(X_train_cleaned, y_train_cleaned):
    svr = SVR()
    gs_svr_params = {
    'kernel': ['poly', 'rbf', 'sigmoid'],
    'C': np.logspace(-1, 2.2, 4),
    'epsilon': np.logspace(-2, 1, 3),
    }
    gs_svr = GridSearchCV(svr, gs_svr_params, cv=5, verbose=3)
    gs_svr.fit(X_train_cleaned, y_train_cleaned)
    print(f"""The best validation score obtained is {gs_svr.best_score_:.5f} with
    \tkernel: {gs_svr.best_params_['kernel']}
    \tC: {gs_svr.best_params_['C']}
    \tepsilon: {gs_svr.best_params_['epsilon']}""")
    return gs_svr

---
## Create CSV for submission

In [17]:
def create_submission(prediction, sub_id, basepath = 'submissions/task1-sub'):
    result = prediction.copy()
    result = result.rename(columns={0: 'y'})
    result['id'] = range(0, len(result))
    result = result[['id', 'y']]
    result.to_csv(basepath+str(sub_id) + '.csv', index=False)

---
## Final Pipeline

In [18]:
print("Loading data\n")
X_train, y_train, X_test = import_data()


print("Removing outliers from x:")
X_train_no_outliers, y_train_no_outliers = remove_outliers(X_train, y_train, 0.02) #Remove outliers in x
print("\nRemoving outliers from y:")
y_train_no_outliers, X_train_no_outliers = remove_outliers(y_train_no_outliers, X_train_no_outliers, contamination=0.005) #Remove outliers in y

print("\nScaling data\n")
X_train_scaled, X_test_scaled= scale(X_train_no_outliers, X_test)

print("\nImputing Nan Values:")
X_train_imputed, X_test_imputed = impute_values(X_train_scaled, X_test_scaled)

print("\nSelecting features:")
X_train_selected_features, X_test_selected_features = select_features(X_train_imputed, X_test_imputed)

Loading data

Removing outliers from x:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


Detected 25 outliers, out of 1212 samples (2.06%).

Removing outliers from y:


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Detected 5 outliers, out of 1187 samples (0.42%).

Scaling data


Imputing Nan Values:
For the train dataset, there are 75010 nan values, out of 983424 (7.63%).

Selecting features:
4 features removed because of constant values (0.48%)
95 features removed because of correlation > 0.8 (11.47%)
8 features removed because of correlation < 0.08 (1.09%) with every other features


In [19]:
gs = model(X_train_selected_features, y_train_no_outliers)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .........................alpha=0.1;, score=0.275 total time=   0.2s
[CV 2/5] END .........................alpha=0.1;, score=0.268 total time=   0.1s
[CV 3/5] END .........................alpha=0.1;, score=0.315 total time=   0.1s
[CV 4/5] END .........................alpha=0.1;, score=0.305 total time=   0.1s
[CV 5/5] END .........................alpha=0.1;, score=0.332 total time=   0.1s
[CV 1/5] END .........alpha=0.11288378916846889;, score=0.299 total time=   0.1s
[CV 2/5] END .........alpha=0.11288378916846889;, score=0.286 total time=   0.1s
[CV 3/5] END .........alpha=0.11288378916846889;, score=0.331 total time=   0.1s
[CV 4/5] END .........alpha=0.11288378916846889;, score=0.323 total time=   0.1s
[CV 5/5] END .........alpha=0.11288378916846889;, score=0.340 total time=   0.1s
[CV 1/5] END .........alpha=0.12742749857031338;, score=0.319 total time=   0.1s
[CV 2/5] END .........alpha=0.12742749857031338

/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 1/5] END ..C=0.1, epsilon=0.01, kernel=poly;, score=0.003 total time=   0.3s


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/5] END .C=0.1, epsilon=0.01, kernel=poly;, score=-0.007 total time=   0.3s


...

[CV 4/5] END C=158.48931924611142, epsilon=10.0, kernel=sigmoid;, score=-73.948 total time=   0.9s


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 5/5] END C=158.48931924611142, epsilon=10.0, kernel=sigmoid;, score=-66.738 total time=   0.9s


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


The best validation score obtained is 0.39818 with
    	kernel: rbf
    	C: 13.593563908785255
    	epsilon: 0.31622776601683794


In [20]:
prediction = pd.DataFrame(gs.predict(X_test_selected_features))
SUB_ID = 6 #to modify
create_submission(prediction, SUB_ID)